In [2]:
from confluent_kafka import Consumer
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Kafka Consumer Configuration
consumer_config = {
    "bootstrap.servers": "pkc-619z3.us-east1.gcp.confluent.cloud:9092",  # Provided bootstrap server
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms": "PLAIN",
    "sasl.username": "CFAIVUXPP46AIDVS",  # Provided API Key
    "sasl.password": "s9bSJiQZ961Bmtvhl7is+lrvlpmmDL4LRykchMqwkVHbEFIAoX3X8jMiHAcY4TTA",  # Provided Secret
    "group.id": "email-consumer-group",  # Consumer group
    "auto.offset.reset": "earliest",  # Start from the earliest message
}

# Kafka topics to listen to
single_topic = "detection_results_single"
batch_topic = "detection_results_single_batch"
consume_topic_name = "emails-topic-new"
def consume_messages():
    """Consumes messages from Kafka topics."""
    consumer = Consumer(consumer_config)
    consumer.subscribe([consume_topic_name])

    print("Listening for messages...")
    try:
        while True:
            msg = consumer.poll(1.0)  # Poll messages every second
            if msg is not None and not msg.error():
                print(f"Topic: {msg.topic()}, Key: {msg.key().decode('utf-8')}, Value: {msg.value().decode('utf-8')}")
    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

if __name__ == "__main__":
    consume_messages()


Listening for messages...


%6|1732999938.680|GETSUBSCRIPTIONS|rdkafka#consumer-2| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to cBakN3ezR0qapPWq+epVBQ


Topic: emails-topic-new, Key: 19370a7793c88c3f, Value: {"id": "19370a7793c88c3f", "snippet": "Google Auth Library was granted access to your Google Account dazzlingdatadude@gmail.com If you did not grant access, you should check this activity and secure your account. Check activity You can also", "from": "Google <no-reply@accounts.google.com>", "subject": "Security alert", "date": "Thu, 28 Nov 2024 02:44:03 GMT", "body": "[image: Google]\r\nGoogle Auth Library was granted access to your Google Account\r\n\r\n\r\ndazzlingdatadude@gmail.com\r\n\r\nIf you did not grant access, you should check this activity and secure your\r\naccount.\r\nCheck activity\r\n<https://accounts.google.com/AccountChooser?Email=dazzlingdatadude@gmail.com&continue=https://myaccount.google.com/alert/nt/1732761843000?rfn%3D127%26rfnc%3D1%26eid%3D6779920641483252125%26et%3D0>\r\nYou can also see security activity at\r\nhttps://myaccount.google.com/notifications\r\nYou received this email to let you know about import

In [1]:
from transformers import pipeline
import re
from urllib.parse import urlparse

# Load the model for phishing detection using Hugging Face's pipeline
model_name = "ealvaradob/bert-finetuned-phishing"
phishing_detector = pipeline('text-classification', model=model_name)

def is_suspicious_sender(sender_email):
    suspicious_domains = ["example.com", "phishing.com", "fraudulentdomain.com"]  # Replace with a real list
    domain = sender_email.split('@')[-1]
    return domain in suspicious_domains

def is_suspicious_url(text):
# Extract all URLs from the text
    urls = re.findall(r'(https?://\S+)', text)
    if not urls:
        return False  # No URLs found, so no suspicion

    # Check each URL using the phishing detection model
    for url in urls:
        # Check if the URL is suspicious using the phishing detection model
        prediction = phishing_detector(url)
        label = prediction[0]['label']
        
        # If model detects it as phishing, flag it as suspicious
        if label == "LABEL_1":  # Adjust if needed based on the model's output
            return True
    return False

def analyze_email(email_body, sender_email):
    # Check if the sender email is suspicious
    sender_suspicious = is_suspicious_sender(sender_email)
    
    # Check if any URL in the email body is suspicious
    url_suspicious = is_suspicious_url(email_body)
    
    # Combine the results
    if sender_suspicious or url_suspicious:
        return "Suspicious"
    else:
        return "Safe"

email_body = """
Dear user, click here to verify your account: https://suspicious-url.com/verify.
If you don't verify, your account will be suspended.
"""

sender_email = "phishing@gmail.com"

# Analyze the email
result = analyze_email(email_body, sender_email)

print(result)  # Will output "Suspicious" if the email contains suspicious URLs or sender


/Users/awwdudee/.pyenv/versions/3.12.7/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# prediction_model.py

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging

# Set up logging for better error reporting
logger = logging.getLogger('model_inference')
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Load the model and tokenizer
MODEL_NAME = "ealvaradob/bert-finetuned-phishing"
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    # Initialize the Hugging Face pipeline
    phishing_detector = pipeline('text-classification', model=model)
    logger.info(f"Model {MODEL_NAME} loaded successfully.")
except Exception as e:
    logger.error(f"Error loading model {MODEL_NAME}: {str(e)}")
    raise

def predict_phishing(text):
    """Predict if the email is phishing or not."""
    try:
        prediction = phishing_detector(text)
        label = prediction[0]['label']  # 'LABEL_0' for Not Phishing, 'LABEL_1' for Phishing
        probability = prediction[0]['score']
        return label, probability
    except Exception as e:
        logger.error(f"Error during prediction: {str(e)}")
        return "Error", 0.0


2024-12-01 17:20:07,623 - ERROR - Error loading model ealvaradob/bert-finetuned-phishing: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.


Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.

In [15]:
# Test model loading outside of pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ealvaradob/bert-finetuned-phishing"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

input_text = "Test email content"
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
print(outputs)


SequenceClassifierOutput(loss=None, logits=tensor([[-4.2507,  4.7214]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch.nn.functional as F

# Load the model and tokenizer
MODEL_NAME = "ealvaradob/bert-finetuned-phishing"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

def predict_phishing(text):
    """Predict if the email is phishing or not."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    
    # Apply softmax to the logits to get probabilities
    probabilities = F.softmax(outputs.logits, dim=-1).detach().numpy()[0]

    # Get the predicted label (index of the highest probability)
    predicted_label_index = torch.argmax(outputs.logits, dim=-1).item()

    # Convert the label index to the corresponding class name
    predicted_label = "Phishing" if predicted_label_index == 1 else "Not Phishing"
    
    return predicted_label, probabilities

# Example usage:
email_text = "Please click this link to verify your account: http://malicious-url.com"
predicted_label, probabilities = predict_phishing(email_text)

print(f"Prediction: {predicted_label}")
print(f"Probabilities: {probabilities}")


Prediction: Phishing
Probabilities: [5.149469e-06 9.999949e-01]


In [ ]:

 # Additional analysis (using BERT model or other analysis)
            result["suspicious_sender"] = is_suspicious_sender(email['from'])
            result["suspicious_content"] = contains_suspicious_content(email['body'])
            result["suspicious_url"] = contains_suspicious_url(email['body'])
